In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import seaborn as sns

from IPython.display import clear_output, display

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
getChar = lambda x: "ABCDEFGHIKLMNOPQRSTUVWXY"[x]

In [ ]:
def prepData(df):
    df = df.copy()
    
    y = df.label
    y = pd.get_dummies(y, drop_first=False)

    X = df.drop('label', axis=1).to_numpy() / 255
    X = X.reshape(-1, 28, 28, 1)
    return X, y

df_train = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
df_test = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

X_train, y_train = prepData(df_train)
X_test, y_test = prepData(df_test)

In [ ]:
index = 123
plt.imshow(X_train[index,:,:], cmap='gray')
print('Letter:', getChar(y_train.iloc[index].argmax()))

# With Keras

In [ ]:
# 80% ish
def Model2DConvolution():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(32, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = 2, padding='valid'))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(24, activation='softmax'))
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

# For PyTorch-comparison:
def miniModel():
    model = Sequential()
    model.add(Flatten(input_shape=(28,28,1)))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(24, activation ='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model = miniModel()
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs = 10, batch_size = 50)

In [ ]:
testIndex = 120

pred = model.predict(X_test[testIndex].reshape(-1, 28,28, 1))
predChar = getChar(pred.argmax())
truth = getChar(y_test.to_numpy().argmax(axis=1)[testIndex])

print('Ground:', truth)
print('Predic:', predChar)
    
plt.imshow(X_test[testIndex].reshape(28, 28), cmap='gray');

In [ ]:
y_pred = model.predict(X_test).argmax(axis=1)
y_true = y_test.to_numpy().argmax(axis=1)

cm = confusion_matrix(y_true, y_pred)

f, ax=plt.subplots(figsize=(15,8))
sns.heatmap(cm,
            annot=True,
            linewidths=0.005,
            linecolor="red",
            fmt=".0f",
            ax=ax,
            xticklabels=[getChar(x) for x in range(24)],
            yticklabels=[getChar(x) for x in range(24)]
           )

plt.xlabel("Prediction")
plt.ylabel("Ground truth")
plt.show()

In [ ]:
print(accuracy_score(y_true, y_pred))

# The "same" with PyTorch:

In [ ]:
def convertToTensor(X, y):
    X = torch.tensor(X.reshape(-1, 28*28)).float()
    y = torch.tensor(y.to_numpy()).float() # multidim categorical
    #y = torch.tensor(y.to_numpy().argmax(axis=1)).long()
    return X, y

X_train_, y_train_ = convertToTensor(X_train, y_train)
X_test_, y_test_ = convertToTensor(X_test, y_test)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()        
        self.l1 = nn.Linear(28*28, 512)
        self.act1 = nn.ReLU()
        self.l2 = nn.Linear(512, 128)
        self.act2 = nn.ReLU()
        self.l3 = nn.Linear(128, 24)

    def forward(self, x):
        x = self.act1(self.l1(x))
        x = self.act2(self.l2(x))
        x = torch.sigmoid(self.l3(x))
        return x
    
model_torch = NeuralNetwork()
print(model_torch)

In [ ]:
lossFn = nn.BCELoss()
#lossFn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model_torch.parameters())
BS = 32
losses, accs = list(), list()
i = 0
while True:
    i += 1
    
    # Batching with random choices: (a bit hacky ...)
    sample = np.random.randint(0, X_train.shape[0], size=BS)
    X = X_train_[sample]
    y = y_train_[sample]
    
    # Prediction + error:
    out = model_torch(X)
    loss = lossFn(out, y)
    
    # Backprop:
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Infos + plotting:
    y = y.argmax(axis=1)
    cats = torch.argmax(out, axis=1)
    acc = (cats == y).float().mean()

    losses.append(loss.item())
    accs.append(acc.item()*100)
    
    if i % 20 == 0:
        test = (model_torch(X_test_).argmax(axis=1) == y_test_.argmax(axis=1)).float().mean().numpy()*100
        print(f"ITERATION: {i}\tACC-TRAIN: {accs[-1]:.1f}\tLOSS: {losses[-1]:.2f}\t ACC-TEST:{test:.2f}")
        clear_output(wait=True)

    # Stop if takes too long:
    if i > 5000:
        print("Stop by max iter!")
        break
        
    # Stop if train-acc is greater than X:
    if np.mean(accs[-100:]) > 98:
        print("> 98%")
        break

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax2 = ax.twinx()

ax.plot(accs, label='acc', color='orange')
ax.legend()

ax2.plot(losses, label='loss', color='red')
ax2.legend()
plt.grid()

In [ ]:
y_pred_ = model_torch(X_test_).detach().numpy().argmax(axis=1)
y_true_ = y_test_.detach().numpy().argmax(axis=1)

cm = confusion_matrix(y_true_, y_pred_)

f, ax=plt.subplots(figsize=(15,8))
sns.heatmap(cm,
            annot=True,
            linewidths=0.005,
            linecolor="red",
            fmt=".0f",
            ax=ax,
            xticklabels=[getChar(x) for x in range(24)],
            yticklabels=[getChar(x) for x in range(24)]
           )

plt.xlabel("Prediction")
plt.ylabel("Ground truth")
plt.show()

In [ ]:
(model_torch(X_test_).argmax(axis=1) == y_test_.argmax(axis=1)).float().mean().numpy()*100